# Student performance (Kaggle)
- https://www.kaggle.com/c/184702-tu-ml-ws-18-student-performance
- small samples (train = 198), medium dimension (32)
- attribute characteristics: numeric, categorical
- Predict: Grade
- Result file cols: id, Grade
- Missing values: No

- import data
- one hot encode (+ drop first columns) categorical data
- 

In [1]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import datetime as dt
%run './base.ipynb'

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [2]:
# TODO: merge train and test with 
# 'dataset = pd.concat([train_data, test_data], keys = ['train', 'test'])', 
# then one hot encode


train = pd.read_csv('./data/student_performance_kaggle/StudentPerformance.shuf.train.csv')
test = pd.read_csv('./data/student_performance_kaggle/StudentPerformance.shuf.test.csv')
#display(test)
#train.info()

train_s, test_s = scale_data(train, test, exclude_train=['Grade', 'id'], exclude_test=['id'])
#display(train_s)
train_oh = one_hot(train_s)
display(train_oh)


#display(train_oh.info(verbose=True))

,id,Grade,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,...,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,70,16,-1.246972,0.206085,-1.406158,0.763565,2.349905,-0.459841,0.033520,0.733048,...,0,0,0,1,0,0,0,1,1,0
1,275,10,0.330667,-0.721298,1.368785,0.763565,-0.011928,-0.459841,0.033520,-0.261086,...,1,0,0,1,1,0,1,1,1,1
2,94,10,-0.458153,1.133468,-0.481177,0.763565,-0.011928,-0.459841,1.139683,-0.261086,...,1,0,0,1,1,1,1,1,1,0
3,276,12,0.330667,-0.721298,-0.481177,0.763565,-0.011928,-0.459841,0.033520,0.733048,...,1,0,1,1,1,0,1,1,1,1
4,241,12,0.330667,1.133468,0.443804,0.763565,-0.011928,-0.459841,-2.178805,1.727183,...,1,0,0,0,0,1,1,1,1,1
5,339,17,1.119486,0.206085,0.443804,-0.649387,2.349905,-0.459841,1.139683,-0.261086,...,1,0,0,1,0,0,1,1,1,0
6,226,7,1.119486,0.206085,-1.406158,-0.649387,-0.011928,0.804722,1.139683,-0.261086,...,1,0,0,0,0,1,1,1,1,1
7,273,11,1.119486,-1.648680,-1.406158,0.763565,-0.011928,-0.459841,0.033520,0.733048,...,1,0,0,1,1,0,0,1,0,0
8,98,10,-0.458153,-0.721298,-1.406158,-0.649387,-0.011928,-0.459841,0.033520,-0.261086,...,1,0,0,1,1,0,1,1,0,1
9,371,9,1.908306,0.206085,-0.481177,0.763565,-0.011928,2.069284,-1.072642,-1.255220,...,0,1,0,0,0,1,1,1,0,1


In [5]:
X_train = train_oh.drop(['Grade', 'id'], axis='columns')
y_train = train_oh[['Grade']]
X_test = test_oh

# params
param_grid = {
    'C': np.linspace(.2,1,5),
    'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], # poly very slow
    'epsilon': np.linspace(0,.5,6),
    'gamma': ['auto']#, 'scale']
}

# run grid search
gs = run_svr(X_train, y_train.values.ravel(), cv=5, param_grid=param_grid)

# predict
result = pd.DataFrame(gs.best_estimator_.predict(X_train))
result.columns = ['Grade']

# join id col
result = pd.concat([train[['id']], result], axis='columns')
display(result)

# Create SVR filename
filename = f'''svr_'''\
           f'''C-{gs.best_estimator_.C}_'''\
           f'''k-{gs.best_estimator_.kernel}_'''\
           f'''e-{gs.best_estimator_.epsilon}_'''\
           f'''g-{gs.best_estimator_.gamma}_'''\
           f'''{dt.datetime.now()}.csv'''

result.to_csv('./predictions/student_performance_kaggle/' + filename, sep = ",", index=False)
print(f'''Saved as {filename}''')

GridSearch initializing...
SVR model in training...
Score: -18.17860225853854, C: 0.2, kernel: linear, epsilon: 0.5, gamma: auto 


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


,id,Grade
0,70,13.475841
1,275,10.500050
2,94,12.233493
3,276,8.680216
4,241,11.768146
5,339,13.857594
6,226,7.976334
7,273,9.773205
8,98,9.499941
9,371,9.499872


Saved as svr_C-0.2_k-linear_e-0.5_g-auto_2018-12-31 01:44:44.700783.csv


In [4]:
def upload_breast_csv(result, filename):

SyntaxError: unexpected EOF while parsing (<ipython-input-4-f1d81033f22f>, line 2)

In [21]:
# Get a feeling for the dataset

# Check if train DataFrame has NaNs
if(train.isnull().values.any()): print('NaNs!')
else: print('Nons!')
    
def boxplots(data):
    for col in data.select_dtypes(['int64', 'float64']).columns:
        fig, ax = plt.subplots()
        bp = ax.boxplot(data[[col]].values)
        ax.set_xlabel(col)
        
#boxplots(train)

def histograms(data):
    for col in data.select_dtypes(['object']).columns:
        fig, ax = plt.subplots()
        hg = ax.hist(data[[col]].values)
        ax.set_xlabel(col)
        
#histograms(train)

Nons!
